In [1]:
import sys
sys.path.append(".")

# also disable grad to save memory
import torch
torch.set_grad_enabled(False)

DEVICE = torch.device("cpu" if torch.cuda.is_available() else "cpu")

import yaml
import torch
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel, GumbelVQ
import io
import os, sys
import requests
import PIL
from PIL import Image
from PIL import ImageDraw, ImageFont
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from nuwa_pytorch import VQGanVAE
from nuwa_pytorch.optimizer import get_optimizer

In [2]:
vae = VQGanVAE(
    dim = 256,
    channels = 1,               # default is 3, but can be changed to any value for the training of the segmentation masks (sketches)
    image_size = 256,           # image size
    num_layers = 4,             # number of downsampling layers
    num_resnet_blocks = 2,      # number of resnet blocks
    vq_codebook_size = 1024,    # codebook size
    vq_decay = 0.8 ,             # codebook exponential decay
    use_hinge_loss = True,
    use_vgg_and_gan = True
).to(DEVICE)
checkpoint = torch.load('/home/hbi/vae_epoch50', map_location = 'cpu')
vae.load_state_dict(checkpoint, strict=False)

<All keys matched successfully>

In [3]:
# Define dataset
import torch
import sys
from nuwa_pytorch import VQGanVAE
import h5py
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision.transforms import ToTensor, Compose, CenterCrop
def eventGeneration(start_time, obs_time = 3 ,lead_time = 6, time_interval = 30):
    # Generate event based on starting time point, return a list: [[t-4,...,t-1,t], [t+1,...,t+72]]
    # Get the start year, month, day, hour, minute
    year = int(start_time[0:4])
    month = int(start_time[4:6])
    day = int(start_time[6:8])
    hour = int(start_time[8:10])
    minute = int(start_time[10:12])
    #print(datetime(year=year, month=month, day=day, hour=hour, minute=minute))
    times = [(datetime(year, month, day, hour, minute) + timedelta(minutes = time_interval * (x+1))) for x in range(lead_time)]
    lead = [dt.strftime('%Y%m%d%H%M') for dt in times]
    times = [(datetime(year, month, day, hour, minute) - timedelta(minutes = time_interval * x)) for x in range(obs_time)]
    obs = [dt.strftime('%Y%m%d%H%M') for dt in times]
    obs.reverse()
    return lead, obs


class radarDataset(Dataset):
    def __init__(self, root_dir, event_times, obs_number = 3, pred_number = 6, transform=None):
        # event_times is an array of starting time t(string)
        # transform is the preprocessing functions
        self.root_dir = root_dir
        self.transform = transform
        self.event_times = event_times
        self.obs_number = obs_number
        self.pred_number = pred_number
    def __len__(self):
        return len(self.event_times)
    def __getitem__(self, idx):
        start_time = str(self.event_times[idx])
        time_list_pre, time_list_obs = eventGeneration(start_time, self.obs_number, self.pred_number)
        output = []
        time_list = time_list_obs + time_list_pre
        #print(time_list)
        for time in time_list:
            year = time[0:4]
            month = time[4:6]
            #path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAC_MFBS_EM_5min_' + time + '_NL.h5'
            path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAP_5min_' + time + '.h5'
            image = np.array(h5py.File(path)['image1']['image_data'])
            #image = np.ma.masked_where(image == 65535, image)

            image = image[264:520,242:498]
            image[image == 65535] = 0
            image = image.astype('float32')
            image = image/100*12
            image = np.clip(image, 0, 128)
            image = image/128
            output.append(image)
        output = torch.permute(torch.tensor(np.array(output)), (1, 2, 0))
        output = self.transform(np.array(output))
        return output, start_time
#root_dir = '/users/hbi/data/RAD_NL25_RAC_MFBS_EM_5min/'
#dataset = radarDataset(root_dir, ["200808031600"], transform = Compose([ToTensor(),CenterCrop(256)]))


In [13]:
# develop dataset
from torch.cuda.amp import autocast
from torch.autograd import Variable
from torch import optim
import pandas as pd
root_dir = '/home/hbi/RAD_NL25_RAP_5min/' 

# M-L, top 5%
df_train = pd.read_csv('training_Delfland08-14.csv', header = None)
event_times = df_train[0].to_list()
dataset_train = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))           

df_test = pd.read_csv('testing_Delfland18-20.csv', header = None)
event_times = df_test[0].to_list()
dataset_test = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_vali = pd.read_csv('validation_Delfland15-17.csv', header = None)
event_times = df_vali[0].to_list()
dataset_vali = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_train_aa = pd.read_csv('training_Aa08-14.csv', header = None)
event_times = df_train_aa[0].to_list()
dataset_train_aa = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))    

df_train_dw = pd.read_csv('training_Dwar08-14.csv', header = None)
event_times = df_train_dw[0].to_list()
dataset_train_dw = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))   

df_train_re = pd.read_csv('training_Regge08-14.csv', header = None)
event_times = df_train_re[0].to_list()
dataset_train_re = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))    

# extreme, top 1%
df_train_ext = pd.read_csv('training_Delfland08-14_ext.csv', header = None)
event_times = df_train_ext[0].to_list()
mfbs = df_train_ext[1].to_list()
dic_mfbs1 = dict(zip(event_times, mfbs))
dataset_train_ext = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))           

df_test_ext = pd.read_csv('testing_Delfland18-20_ext.csv', header = None)
event_times = df_test_ext[0].to_list()
mfbs = df_test_ext[1].to_list()
dic_mfbs2 = dict(zip(event_times, mfbs))
dataset_test_ext = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_vali_ext = pd.read_csv('validation_Delfland15-17_ext.csv', header = None)
event_times = df_vali_ext[0].to_list()
mfbs = df_vali_ext[1].to_list()
dic_mfbs3 = dict(zip(event_times, mfbs))
dataset_vali_ext = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

dic_mfbs = dic_mfbs1 | dic_mfbs2 | dic_mfbs3
new_valid = ['201903070300', '201903070315', '201904241715', '201904241730', '201904241745', '201904241800', '201904241815', '201904241830', '201904241845', '201904241900', '201904241915', '201905280330', '201905280345', '201905280400', '201905280415', '201905280430', '201905280445', '201905280500', '201905280515', '201906052035', '201906052050', '201906052100', '201906120545', '201906120600', '201906120615', '201906120630', '201906120645', '201906120700', '201906120715', '201906120730', '201906120745', '201906120800', '201906120815', '201906120830', '201906120845', '201906120900', '201906120915', '201906120930', '201906120945', '201906121000', '201906121015', '201906121030', '201906121045', '201906121100', '201906150035', '201906150050', '201906150100', '201906150115', '201906150130', '201906150145', '201906150200', '201906150215', '201906150230', '201906150245', '201906150300', '201906150315', '201906150330', '201906190400', '201906190415', '201906190430', '201906190445', '201906190500', '201906190515', '201906190530', '201906190545', '201906190600', '201908121840', '201910210250', '201910210300', '201910210315', '201910210330', '201910210345', '201910210400', '201910210415', '201910210430', '201910210445', '201911280500', '201911280515', '201911280530', '201911280545', '201911280600', '201911280615', '202002091800', '202002091815', '202002091830', '202002091845', '202002091900', '202002161520', '202002161535', '202002161550', '202003051615', '202003051630', '202003051645', '202003051700', '202006050400', '202006050415', '202006050430', '202006050445', '202006050500', '202006050515', '202006050530', '202006050545', '202006050600', '202006050615', '202006050630', '202006121915', '202006121930', '202006121945', '202006122000', '202006122015', '202006122030', '202006122045', '202006161035', '202006161050', '202006161100', '202006161115', '202006161130', '202006161145', '202006161200', '202006161215', '202006161230', '202006161245', '202006171730', '202006171745', '202006171800', '202006171815', '202006171830', '202006171845', '202006171900', '202006171915', '202006171930', '202006171945', '202006172000', '202006172015', '202007252020', '202007252035', '202007252050', '202007252100', '202007252115', '202008161345', '202008161400', '202008161415', '202008161430', '202008161445', '202008161500', '202008161515', '202008161530', '202008161545', '202008161600', '202008161615', '202009232000', '202009232015', '202102031100', '202102031115', '202102031130', '202105131415', '202105131430', '202105131445', '202105131500', '202105131515', '202105131530', '202105131545', '202105131600', '202105131615', '202105131630', '202105131645', '202105131700', '202105131715', '202106180100', '202106180115', '202106180130', '202106180145', '202106180200', '202106192015', '202106192030', '202106192045', '202106192100', '202106192115', '202106192130', '202106192145', '202106192200', '202106192215', '202106192230', '202106192245']
new_valid = ['201903070300', '201903070315', '201904241715', '201904241730', '201904241745', '201904241800', '201904241815', '201904241830', '201904241845', '201904241900', '201904241915', '201905280330', '201905280345', '201905280400', '201905280415', '201905280430', '201905280445', '201905280500', '201905280515', '201906052035', '201906052050', '201906052100', '201906120545', '201906120600', '201906120615', '201906120630', '201906120645', '201906120700', '201906120715', '201906120730', '201906120745', '201906120800', '201906120815', '201906120830', '201906120845', '201906120900', '201906120915', '201906120930', '201906120945', '201906121000', '201906121015', '201906121030', '201906121045', '201906121100', '201906150035', '201906150050', '201906150100', '201906150115', '201906150130', '201906150145', '201906150200', '201906150215', '201906150230', '201906150245', '201906150300', '201906150315', '201906150330', '201906190400', '201906190415', '201906190430', '201906190445', '201906190500', '201906190515', '201906190530', '201906190545', '201906190600', '201908121840', '201910210250', '201910210300', '201910210315', '201910210330', '201910210345', '201910210400', '201910210415', '201910210430', '201910210445', '201911280500', '201911280515', '201911280530', '201911280545', '201911280600', '201911280615', '202002091800', '202002091815', '202002091830', '202002091845', '202002091900', '202002161520', '202002161535', '202002161550', '202003051615', '202003051630', '202003051645', '202003051700', '202006050400', '202006050415', '202006050430', '202006050445', '202006050500', '202006050515', '202006050530', '202006050545', '202006050600', '202006050615', '202006050630', '202006121915', '202006121930', '202006121945', '202006122000', '202006122015', '202006122030', '202006122045', '202006161035', '202006161050', '202006161100', '202006161115', '202006161130', '202006161145', '202006161200', '202006161215', '202006161230', '202006161245', '202006171730', '202006171745', '202006171800', '202006171815', '202006171830', '202006171845', '202006171900', '202006171915', '202006171930', '202006171945', '202006172000', '202006172015', '202007252020', '202007252035', '202007252050', '202007252100', '202007252115', '202008161345', '202008161400', '202008161415', '202008161430', '202008161445', '202008161500', '202008161515', '202008161530', '202008161545', '202008161600', '202008161615', '202009232000', '202009232015', '202102031100', '202102031115', '202102031130', '202105131415', '202105131430', '202105131445', '202105131500', '202105131515', '202105131530', '202105131545', '202105131600', '202105131615', '202105131630', '202105131645', '202105131700', '202105131715', '202106180100', '202106180115', '202106180130', '202106180145', '202106180200', '202106192015', '202106192030', '202106192045', '202106192100', '202106192115', '202106192130', '202106192145', '202106192200', '202106192215', '202106192230', '202106192245', '201904241800', '201904241815', '201904241830', '201907111120', '201907111135', '201907111150', '201907111200', '201907111215', '201907111230', '201907111245', '201907111300', '201907111315', '201907111330', '201908091035', '201908091050', '201908091100', '201908091115', '201908091130', '201908091145', '201908091200', '201908121410', '201908121425', '201908121440', '201908121500', '201908121515', '201908121530', '201908121545', '201908121600', '201908121615', '201908121630', '201908121645', '201908282300', '201908282315', '201908282330', '201908282345', '201908290000', '201908290015', '201908290030', '201908290045', '201908290100', '201908290115', '201908290130', '201908290145', '201909261805', '201909261820', '201909261835', '201909261850', '201909261900', '201909261915', '201909261930', '201909261945', '201909262000', '201910011820', '201910011835', '201910011850', '201910011900', '201910011915', '201910011930', '201910011945', '201910012000', '201910012015', '201910012030', '202006141135', '202006141150', '202006141200', '202006141215', '202006141230', '202006141245', '202006141300', '202006141315', '202006141330', '202006141345', '202006141400', '202006141415', '202006141430', '202006141445', '202006141500', '202006141515', '202006141530', '202006141545', '202006141600', '202006141615', '202006141630', '202006141645', '202006141700', '202006141715', '202006141730', '201904241750', '201904241800', '201905191600', '201905191615', '201905191630', '201905191645', '201905191700', '201905191715', '201905191730', '201905191745', '201906052135', '201906052150', '201906052200', '201906052215', '201906150100', '201906150115', '201906150130', '201906150145', '201907261340',  '201907261415', '201907261430', '201907261445', '201907261500', '201907261515', '201908282240', '201908282300', '201908282315', '201908282330', '201908282345', '201908290000', '201908290015', '201908290030', '201908290045', '201908290100', '201908290115', '201910010435', '201910010450', '201910010500', '201910010515', '201910010530', '201910010545', '201910010600', '201910200930', '201910200945', '202006122010', '202006122025', '202006122040', '202006122100', '202006122115', '202006122130', '202006122145', '202006122200', '202006122215', '202006122230', '202006122245', '202006171525', '202006171540', '202006171600', '202006171615', '202006171630', '202006171645', '202006171700', '202006171715', '202006171730', '202006171745', '202006171800', '202006171815', '202006171830', '202006171845', '202006261700', '202006261715', '202006261730', '202006261745', '202006261800', '202006261815', '202006261830', '202006261845', '202006261900', '202006261915', '202006261930', '202006261945', '202006262000', '202009261920', '202009261935', '202009261950', '202009262000']

print(len(new_valid))

dataset_ext = radarDataset(root_dir, new_valid, transform = Compose([ToTensor()])) 

#print("Extreme:", len(dataset_train_ext), len(dataset_test_ext), len(dataset_vali_ext))
loaders = { 'test' :DataLoader(dataset_test_ext, batch_size=1, shuffle=True, num_workers=8), 
           'valid' :DataLoader(dataset_vali, batch_size=1, shuffle=False, num_workers=0),
            'ext' :DataLoader(dataset_ext, batch_size=1, shuffle=False, num_workers=8),
           'train_ext' :DataLoader(dataset_train_ext, batch_size=1, shuffle=True, num_workers=8),
           'train_aa5' :DataLoader(dataset_train_aa, batch_size=1, shuffle=False, num_workers=0),
           'train_del5' :DataLoader(dataset_train, batch_size=1, shuffle=True, num_workers=0),
           'train_dw5' :DataLoader(dataset_train_dw, batch_size=1, shuffle=False, num_workers=0),
           'train_re5' :DataLoader(dataset_train_re, batch_size=1, shuffle=False, num_workers=0)}

#print(len(loaders['ext']))
#print(dataset_ext[0])

357


In [14]:
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
from nuwa_pytorch.nuwa_pytorch import top_k, gumbel_sample
from collections import Counter
import time
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
from pysteps.postprocessing.probmatching import nonparam_match_empirical_cdf
from tqdm import tqdm
import math
import random
device = DEVICE
loaders = { 'test' :DataLoader(dataset_test_ext, batch_size=1, shuffle=True, num_workers=8), 
           'valid' :DataLoader(dataset_vali, batch_size=1, shuffle=False, num_workers=0),
            'ext' :DataLoader(dataset_ext, batch_size=1, shuffle=False, num_workers=8),
           'train_ext' :DataLoader(dataset_train_ext, batch_size=1, shuffle=True, num_workers=8),
           'train_aa5' :DataLoader(dataset_train_aa, batch_size=1, shuffle=False, num_workers=0),
           'train_del5' :DataLoader(dataset_train, batch_size=1, shuffle=True, num_workers=0),
           'train_dw5' :DataLoader(dataset_train_dw, batch_size=1, shuffle=False, num_workers=0),
           'train_re5' :DataLoader(dataset_train_re, batch_size=1, shuffle=False, num_workers=0)}
#torch.cuda.empty_cache()

In [18]:
import cv2
from pysteps.postprocessing.probmatching import nonparam_match_empirical_cdf
from statistics import mean
number = 5
for name in ['pystep']:
#'/home/hbi/predictions_new_evl', '/home/hbi/predictions_new_ce', '/home/hbi/predictions_new_wce']:
#for name in ['/home/hbi/predictions_ce','/home/hbi/predictions_evl100', '/home/hbi/predictions_wce',
#for name in ['/home/hbi/predictions_evl100_fa3']:
    pcc_sum, mse_sum, mae_sum, csi1_sum, csi2_sum, csi8_sum, far1_sum, far2_sum, far8_sum, fss1_sum, fss10_sum, fss20_sum, fss30_sum, hr, fa = [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6, [0]*6
    count = 0
    count_nan = 0
    count_list = [0]*6
    print(name)
    for i, (images,time) in enumerate(loaders['ext']):
        print(i, time[0], end = '\r')
        image = images[0].unsqueeze(1).unsqueeze(0)
        a = Variable(image).to(DEVICE)   # batch x
        a1 = a.squeeze(0)
        # Generate
        count+=1
        n = 6
        
        if name == 'pystep':
            vp = np.array(h5py.File("/home/hbi/predictions_ensamble/" + time[0] + "_nowcast_small20_delfland.nc")['Prediction']["Pre"])
            #vp = np.array(h5py.File("/home/hbi/predictions_pysteps/" + time[0] + "_nowcast_ensamble_delfland.nc")['Prediction']["Pre"])[:,264:520,242:498]
            #vp = np.array(h5py.File("/home/hbi/predictions_new_pysteps/" + time[0] + "_nowcast_ensamble_delfland.nc")['Prediction']["Pre"])[:,264:520,242:498]
            vp[vp<0] = 0
            vp = vp.astype('float64')
            vp = vp[:5, :, :, :]
            vp = np.mean(vp, axis=0)
        else:
            for k in range(20):
                if k==0:
                    vp = np.load(name+'_1/{}.npy'.format(time[0]))
                    #for j in range(6):
                        #vp[j] = vp[j]*(1+0.66*((vp[j]/np.max(vp[j]))**0.81))
                        #vp[j] = nonparam_match_empirical_cdf(vp[j], a1[2,0,:,:].to('cpu').detach().numpy()*40)
                    vp[np.isnan(vp)]=0
                else:
                    try:
                        vp_temp = np.load(name+'_{}/{}.npy'.format(k+1, time[0]))
                    except:
                        vp_temp = np.load('/home/hbi/predictions_new_ce'+'_{}/{}.npy'.format(k+1, time[0]))
                    #for j in range(6):
                        #vp_temp[j] = vp_temp[j]*(1+0.66*((vp_temp[j]/np.max(vp_temp[j]))**0.81))
                        #vp_temp[j] = nonparam_match_empirical_cdf(vp_temp[j], a1[2,0,:,:].to('cpu').detach().numpy()*40)
                    vp_temp[np.isnan(vp_temp)]=0
                    vp += vp_temp
            vp/=20
            for j in range(6):
                vp[j] = vp[j]*(1+0.66*((vp[j]/np.max(vp[j]))**0.81))
        flag = 0
        for t in range(n):
            a1_display = a1[t+3,0,:,:].to('cpu').detach().numpy()*40
            a_p1_display = vp[t,:,:]
            a_p1_display[a_p1_display<=0]=0
            a_p1_display[np.isnan(a_p1_display)]=0

            #a1_display = cv2.resize(a1_display, (512,512), interpolation = cv2.INTER_CUBIC)
            #a_p1_display = cv2.resize(a_p1_display, (512,512), interpolation = cv2.INTER_CUBIC)
            #a_p1_display = a_p1_display*(1+0.66*((a_p1_display/np.max(a_p1_display))**0.81))
            #a_p1_display = nonparam_match_empirical_cdf(a_p1_display, a1[2,0,:,:].to('cpu').detach().numpy()*40)
            if False:
                plt.figure(figsize=(12, 4))
                plt.subplot(131)
                plot_precip_field(a1_display, title="t+{}".format((t+1)*30), axis='off')
                plt.subplot(132)
                plot_precip_field(a_p1_display, title="Prediction")
                plt.tight_layout()
                plt.show()

            scores_cont1 = det_cont_fct(a_p1_display, a1_display, thr=0.1)
            scores_cat1 = det_cat_fct(a_p1_display, a1_display, 1)
            scores_cat2 = det_cat_fct(a_p1_display, a1_display, 2)
            scores_cat8 = det_cat_fct(a_p1_display, a1_display, 8)
            scores_spatial = intensity_scale(a_p1_display, a1_display, 'FSS', 1, [1,10,20,30])

            try:
                #if not (math.isnan(scores_cat1['CSI']) or math.isnan(scores_cat1['FAR']) or math.isnan(scores_cat2['CSI']) or math.isnan(scores_cat2['FAR']) or math.isnan(scores_cat8['CSI']) or math.isnan(scores_cat8['FAR']) or math.isnan(scores_spatial[3][0]) or math.isnan(scores_spatial[2][0]) or math.isnan(scores_spatial[1][0]) or math.isnan(scores_spatial[0][0])):
                pcc_sum[t] += scores_cont1['corr_p']
                mse_sum[t] += scores_cont1['MSE']
                mae_sum[t] += scores_cont1['MAE']
                if math.isnan(scores_cat8['CSI']):
                    csi8_sum[t] += 0
                else:
                    csi8_sum[t] += scores_cat8['CSI']
                if math.isnan(scores_cat1['FAR']):
                    far1_sum[t] += 0
                else:
                    far1_sum[t] += scores_cat1['FAR']
                if math.isnan(scores_cat2['FAR']):
                    far2_sum[t] += 0
                else:
                    far2_sum[t] += scores_cat2['FAR']
                if math.isnan(scores_cat8['FAR']):
                    far8_sum[t] += 0
                else:
                    far8_sum[t] += scores_cat8['FAR']
                csi1_sum[t] += scores_cat1['CSI']
                csi2_sum[t] += scores_cat2['CSI']            
                fss1_sum[t] += scores_spatial[3][0]
                fss10_sum[t] += scores_spatial[2][0]
                fss20_sum[t] += scores_spatial[1][0]
                fss30_sum[t] += scores_spatial[0][0]
                count_list[t] += 1
            except:
                continue

    for t in range(6):
        pcc_sum[t] = round(pcc_sum[t]/count_list[t],5)
        mse_sum[t] = round(mse_sum[t]/count_list[t],5)
        mae_sum[t] = round(mae_sum[t]/count_list[t],5)
        csi1_sum[t] = round(csi1_sum[t]/count_list[t],5)
        csi2_sum[t] = round(csi2_sum[t]/count_list[t],5)
        csi8_sum[t] = round(csi8_sum[t]/count_list[t],5)
        far1_sum[t] = round(far1_sum[t]/count_list[t],5)
        far2_sum[t] = round(far2_sum[t]/count_list[t],5)
        far8_sum[t] = round(far8_sum[t]/count_list[t],5)
        fss1_sum[t] = round(fss1_sum[t]/count_list[t],5)
        fss10_sum[t] = round(fss10_sum[t]/count_list[t],5)
        fss20_sum[t] = round(fss20_sum[t]/count_list[t],5)
        fss30_sum[t] = round(fss30_sum[t]/count_list[t],5)
    print('pcc =', pcc_sum, round(mean(pcc_sum),4))
    print('mse =', mse_sum, round(mean(mse_sum),4))
    print('mae =', mae_sum, round(mean(mae_sum),4))
    print('csi1 =', csi1_sum, round(mean(csi1_sum),4))
    print('csi2 =', csi2_sum, round(mean(csi2_sum),4))
    print('csi8 =', csi8_sum, round(mean(csi8_sum),4))
    print('FAR_1 =', far1_sum, round(mean(far1_sum),4))
    print('FAR_2 =', far2_sum, round(mean(far2_sum),4))
    print('FAR_8 =', far8_sum, round(mean(far8_sum),4))
    print('fss1 =', fss1_sum, round(mean(fss1_sum),4))
    print('fss10 =', fss10_sum, round(mean(fss10_sum),4))
    print('fss20 =', fss20_sum, round(mean(fss20_sum),4))
    print('fss30 =', fss30_sum, round(mean(fss30_sum),4))
         

pystep
pcc = [0.43897, 0.28688, 0.20703, 0.14614, 0.11071, 0.09343] 0.2139
mse = [4.06656, 4.42433, 4.43701, 4.06908, 3.92954, 3.51296] 4.0732
mae = [0.66666, 0.80073, 0.87699, 0.88581, 0.89035, 0.83131] 0.8253
csi1 = [0.44986, 0.32682, 0.25066, 0.19047, 0.15272, 0.12466] 0.2492
csi2 = [0.29195, 0.17939, 0.12461, 0.08464, 0.0627, 0.0487] 0.132
csi8 = [0.03139, 0.00764, 0.00382, 0.00427, 0.00182, 0.00227] 0.0085
FAR_1 = [0.38366, 0.47172, 0.51477, 0.57331, 0.60921, 0.64499] 0.5329
FAR_2 = [0.51231, 0.61718, 0.66913, 0.71949, 0.74308, 0.78621] 0.6746
FAR_8 = [0.62373, 0.64346, 0.63025, 0.57441, 0.54202, 0.5401] 0.5923
fss1 = [0.60945, 0.47989, 0.38945, 0.30782, 0.25331, 0.20963] 0.3749
fss10 = [0.73822, 0.59127, 0.48731, 0.3903, 0.32344, 0.27078] 0.4669
fss20 = [0.80136, 0.65804, 0.54906, 0.44479, 0.36941, 0.31053] 0.5222
fss30 = [0.83866, 0.70491, 0.59522, 0.48609, 0.40494, 0.34152] 0.5619
